<a href="https://colab.research.google.com/github/Fakkiie/pathology-slides/blob/main/tesspractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies

In [ ]:
!sudo apt-get update
!sudo apt-get install libzbar0
!sudo apt install tesseract-ocr
!pip install datasets pydicom matplotlib opencv-python pytesseract pyzbar openslide-python tifffile
!pip install openslide-bin

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

### Imports


In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pytesseract
import pydicom
import openslide
from PIL import Image
from pyzbar.pyzbar import decode


### Testing Code
I'm applying Tesseract to multiple DICOM & SVS Files


In [ ]:
def extract_text_from_image(image_path): #Apply OCR to extract text from an image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text.strip()

def process_dicom(dicom_path, output_dir): #Extract burned in text from DICOM file
    filename = os.path.basename(dicom_path).replace(".dcm","")
    dicom_output_dir = os.path.join(output_dir, filename)
    os.makedirs(dicom_output_dir, exist_ok=True)
    dcm = pydicom.dcmread(dicom_path) #Load DICOM file

    image_path = os.path.join(dicom_output_dir, "dicom_image.png")
    if hasattr(dcm, "pixel_array"):
      img = dcm.pixel_array
      img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
      cv2.imwrite(image_path, img)

      with open(os.path.join(dicom_output_dir, "extracted_text.txt"), "w") as f:
          f.write(ocr_text)
    print(f"Processed DICOM file: {dicom_path}")

def process_svs(svs_path, output_dir): #Extract burned in text from SVS file
   filename = os.path.basename(svs_path).replace(".svs","")
   svs_output_dir = os.path.join(output_dir, filename)
   os.makedirs(svs_output_dir, exist_ok=True)

   slide = open.slide.OpenSlide(svs_path)

   label_path = os.path.join(svs_output_dir, "svs_label.png") #Extract label image if possible
   if "label" in slide.associated_images:
       label = slide.associated_images["label"]
       label.save(label_path)
       ocr_text = extract_text_from_image(label_path)

       with open(os.path.join(svs_output_dir, "label_text.txt"), "w") as f:
           f.write(ocr_text)
       print(f"Processed SVS file: {svs_path}")

def process_directory(dicom_dir,svs_dir, output_dir): #Process all DICOM & SVS files in a directory
    for filename in os.listdir(dicom_dir):
      if file.endswith(".dcm"):
        process_dicom(os.path.join(dicom_dir, filename), output_dir)

    for filename in os.listdir(svs_dir):
      if file.endswith(".svs"):
        process_svs(os.path.join(svs_dir, filename), output_dir)
    print(f"Processed all files in directory: {directory_path}")


